# Making line plots

### Import packages

In [1]:
import numpy                          as np
import matplotlib.pyplot              as plt
import matplotlib.animation           as ani
import IPython
import os
%matplotlib agg

# import plons scripts
import plons
import plons.SmoothingKernelScript    as sk
import plons.PhysicalQuantities       as pq
import plons.ConversionFactors_cgs    as cgs
import plons.Plotting                 as plot

### Setting information about data

In [2]:
prefix = "wind"
loc = "Model"
dumpnumber = 600
dump = os.path.join(loc, f"{prefix}_{dumpnumber:06d}")

### Retrieving the data

Creating the directory to store the models

In [3]:
!mkdir -p $loc

Downloading the data

In [4]:
%%capture
if not os.path.exists(dump):
    !wget "https://github.com/Ensor-code/phantom-models/raw/refs/heads/main/Esseldeurs+2023/BinaryHighLucy/wind_00600?download=" --output-document $dump
infile = os.path.join(loc, f"{prefix}.in")
if not os.path.exists(infile):
    !wget "https://raw.githubusercontent.com/Ensor-code/phantom-models/refs/heads/main/Esseldeurs%2B2023/BinaryHighLucy/wind.in" --output-document $infile
setupfile = os.path.join(loc, f"{prefix}.setup")
if not os.path.exists(setupfile):
    !wget "https://raw.githubusercontent.com/Ensor-code/phantom-models/refs/heads/main/Esseldeurs%2B2023/BinaryHighLucy/wind.setup" --output-document $setupfile

### Loading setup and dump

In [5]:
setup     = plons.LoadSetup(loc, prefix)

In [6]:
dumpData  = plons.LoadFullDump(dump, setup)

## Animation

I will create an animation with a 2D slice plot on the left, and a line slice on the right.

In [7]:
fig = plt.figure(figsize=(6, 3))

ax = [None, None]
ax[0] = plt.subplot2grid((1, 2), (0, 0))
ax[1] = plt.subplot2grid((1, 2), (0, 1), aspect=ax[0].get_aspect())

#### Creating 2D slice plot on the left

In [8]:
n = 500
x = np.linspace(-30*cgs.au, 30*cgs.au, n)
y = np.linspace(-30*cgs.au, 30*cgs.au, n)
X, Y = np.meshgrid(x, y)
Z    = np.zeros_like(X)
theta = pq.getPolarAngleCompanion(dumpData._params['posComp'][0], dumpData._params['posComp'][1])
X_rot, Y_rot, Z_rot = sk.rotateMeshAroundZ(theta, X, Y, Z)
smooth_rot = sk.smoothMesh(X_rot, Y_rot, Z_rot, dumpData, ['rho'])

ax[0].set_aspect('equal')
ax[0].set_facecolor('k')

ax[0].pcolormesh(X/cgs.au, Y/cgs.au, np.log10(smooth_rot["rho"]+1e-99), cmap=plt.colormaps['inferno'], vmin=-17, vmax = -14)
ax[0].set_xlim(x[0]/cgs.au, x[-1]/cgs.au)
ax[0].set_ylim(y[0]/cgs.au, y[-1]/cgs.au)

plot.plotSink(ax[0], dumpData, setup, rotate=True)

(<matplotlib.patches.Circle at 0x7ad4272c7f10>,
 <matplotlib.patches.Circle at 0x7ad42fa7bfd0>)

#### Creating the line slices on the right

Create a line plot, like in example 1. At the end, give the axis the same shape as the slice 2D plot

In [9]:
n = 100
rx0 = np.array((np.linspace(0, 50, n), np.zeros(n), np.zeros(n))).transpose()*cgs.au
theta0 = pq.getPolarAngleCompanion(dumpData._params['posComp'][0], dumpData._params['posComp'][1])
rz = np.array((np.zeros(n), np.zeros(n), np.linspace(0, 50, n))).transpose()*cgs.au

line0 = ax[0].plot(rx0[:,0], rx0[:,1], color="white")[0]

smoothx = sk.getSmoothingKernelledPix(10, dumpData, ['rho'], sk.rotatePixCoordAroundZ(theta0, rx0))
smoothz = sk.getSmoothingKernelledPix(10, dumpData, ['rho'], rz)

line1 = ax[1].plot(rx0[:,0]/cgs.au, np.log10(smoothx["rho"]+1e-99))[0]
ax[1].plot(rz[:,2]/cgs.au, np.log10(smoothz["rho"]+1e-99))
ax[1].set_xlim(0, 50)
ax[1].set_ylim(-18, -13)

posx0 = ax[0].get_position()
posx1 = ax[1].get_position()
ax[1].set_position([posx1.x0, posx0.y0, posx0.width, posx0.height])

#### Defining what to update in each frame

In each frame, an angle theta will be returned. Create a line at this angle, calculate the smoothed values, and plot them on the figures

In [10]:
def animate(theta):
    rx = sk.rotatePixCoordAroundZ(-theta, rx0)
    smoothx = sk.getSmoothingKernelledPix(100, dumpData, ['rho'], sk.rotatePixCoordAroundZ(theta0, rx))

    line0.set_data(rx[:,0], rx[:,1])
    line1.set_data(rx0[:,0]/cgs.au, np.log10(smoothx["rho"]+1e-99))

#### Creating the animation

By creating the animation, the frames will represent the angles from the previous function. Interval represents the delay between frames in milliseconds.

In [11]:
anim = ani.FuncAnimation(fig, animate, frames=np.linspace(0, 2*np.pi, 500, endpoint=False), interval=20)
IPython.display.display(IPython.display.HTML(anim.to_html5_video()))
plt.close()